# NormMUSIC Demo

The goal of this notebook is to demonstrate the effect of frequency normalization when MUSIC is applied on broadband signals

The notebook is structured as follows:
1. [Dataset generation](#dataset)
2. [Prediction](#prediction)
    <ol>
        <li>MUSIC: Standard implementation without normalization</li>
        <li>NormMUSIC: Implementation with frequency normalization as suggested in [1]</li>
    </ol>
3. [Evaluation](#evaluation)
4. [Intuition: Why normalization?](#intuition)
4. [Recommendation](#recommendation)
--- 
[1] D. Salvati, C. Drioli and G. L. Foresti, "Incoherent Frequency Fusion for Broadband Steered Response Power Algorithms in
Noisy Environments," in IEEE Signal Processing Letters, vol. 21, no. 5, pp. 581-585, 2014.

In [ ]:
# imports
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import stft
from random import uniform, sample
from pyroomacoustics import doa, Room, ShoeBox

<a id='dataset'></a>
## Dataset generation
In the following, we simulate a small dataset to evaluate the performance of MUSIC and NormMUSIC.
We assume a single sound source.
- Simulate different rooms:
 - DOA on 1° grid
 - 3 samples per DOA
 - Source signal: Random Gaussian
 - Different SNRs between 0 and 30 dB
- Calculate 30 STFT time frames for each sample

In [ ]:
# constants / config
fs = 16000 
nfft = 1024
n = 5*fs # simulation length of source signal (3 seconds)
n_frames = 30
max_order = 10
doas_deg = np.linspace(start=0, stop=359, num=360, endpoint=True)
rs = [0.5, 1, 1.5]
mic_center = np.c_[[2,2,1]]
mic_locs = mic_center + np.c_[[ 0.04,  0.0, 0.0],
                              [ 0.0,  0.04, 0.0],
                              [-0.04,  0.0, 0.0],
                              [ 0.0, -0.04, 0.0],
]
snr_lb, snr_ub = 0, 30

In [ ]:
# room simulation
data = []
for r in rs:
    for i, doa_deg in enumerate(doas_deg):
        doa_rad = np.deg2rad(doa_deg)
        source_loc = mic_center[:,0] + np.c_[r*np.cos(doa_rad), r*np.sin(doa_rad), 0][0]
        room_dim = [uniform(4,6), uniform(4,6), uniform(2, 4)] # meters

        room = ShoeBox(room_dim, fs=fs, max_order=max_order)
        room.add_source(source_loc, signal=np.random.random(n))
        room.add_microphone_array(mic_locs)
        room.simulate(snr=uniform(snr_lb, snr_ub))
        signals = room.mic_array.signals

        # calculate n_frames stft frames starting at 1 second
        stft_signals = stft(signals[:,fs:fs+n_frames*nfft], fs=fs, nperseg=nfft, noverlap=0, boundary=None)[2]
        data.append([r, doa_deg, stft_signals])

<a id='prediction'></a>
## Prediction
In the following, we apply MUSIC and NormMUSIC to the simulated dataset.

The results are stored in a pandas DataFrame

In [ ]:
kwargs = {'L': mic_locs,
          'fs': fs, 
          'nfft': nfft,
          'azimuth': np.deg2rad(np.arange(360))
}
algorithms = {
    'MUSIC': doa.music.MUSIC(**kwargs),
    'NormMUSIC': doa.normmusic.NormMUSIC(**kwargs),
}
columns = ["r", "DOA"] + list(algorithms.keys())

In [ ]:
predictions = {n:[] for n in columns}
for r, doa_deg, stft_signals in data:
    predictions['r'].append(r)
    predictions['DOA'].append(doa_deg)
    for algo_name, algo in algorithms.items():
        algo.locate_sources(stft_signals)
        predictions[algo_name].append(np.rad2deg(algo.azimuth_recon[0]))
df = pd.DataFrame.from_dict(predictions)

<a id='evaluation'></a>
## Evaluation
In the next cells we calculate the following metrics:
- Mean Absolute Error (MAE)
- Median Absolute error (MEDAE)

In [ ]:
MAE, MEDAE = {}, {}

def calc_ae(a,b):
    x = np.abs(a-b)
    return np.min(np.array((x, np.abs(360-x))), axis=0)

for algo_name in algorithms.keys():
    ae = calc_ae(df.loc[:,["DOA"]].to_numpy(), df.loc[:,[algo_name]].to_numpy())
    MAE[algo_name] = np.mean(ae)
    MEDAE[algo_name] = np.median(ae)


In [ ]:
print(f"MAE\t MUSIC: {MAE['MUSIC']:5.2f}\t NormMUSIC: {MAE['NormMUSIC']:5.2f}")
print(f"MEDAE\t MUSIC: {MEDAE['MUSIC']:5.2f}\t NormMUSIC: {MEDAE['NormMUSIC']:5.2f}")

<a id='intuition'></a>
## Intuition: Why normalization?

### MUSIC revisited: Complex narrowband signals
Before we discuss the intution behind frequency normalization, let's revisit the MUSIC algorithm for complex narrowband signals.

#### The MUSIC pseudo spectrum

The MUSIC pseudo spectrum $\hat{P}_{MUSIC}(\theta)$ is defined as:

$$\hat{P}_{MUSIC}(\mathbf{e}(\theta)) = \frac{1}{\sum_{i=p+1}^{N} |\mathbf{e}(\theta)^H \mathbf{v}_i|}$$,
where
- $\mathbf{v}_i$ are the noise eigenvectors
- $\mathbf{e}(\theta)$ is the candidate steering vector
- $\theta$ is the candidate DOA

MUSIC obtains its estimated DOA $\hat{\theta}$ by maximizing the pseudo spectrum $\hat{P}_{MUSIC}(\theta)$ over the candidate DOAs $\theta \in \{\theta_1, \theta_2, ... \theta_I\}$, i.e.,
$$\hat{\theta} = {arg max}_\theta \;\hat{P}_{MUSIC}(\theta)$$

#### Orthogonality
The main property that is exploited by MUSIC is the orthogonality between the noise eigenvectors $\mathbf{v}_i$ and the steering vector $\mathbf{e}(\theta^{\star})$ of the DOA $\theta^{\star}$, i.e., 
$$ \mathbf{e}(\theta^{\star}) \perp span(\mathbf{v}_{p+1}, \mathbf{v}_{p+2}, \mathbf{v}_{N}) \;\; \Leftrightarrow \;\; |\mathbf{e}(\theta^{\star})^H \mathbf{v}_i| = 0 \;\; \forall i \in \{p+1, p+2, ... N\}$$

In practice, $\hat{\theta}$ is approximately orthogonal to the noise eigenvectors $\mathbf{v}_i$, i.e.,
$$ |\mathbf{e}(\hat{\theta})^H \mathbf{v}_i| \approx 0 \;\; \forall i \in \{p+1, p+2, ... N\}$$

Therefore, ${max}_{\theta} \; \hat{P}_{MUSIC}(\theta) = \frac{1}{\epsilon}$, with $\epsilon \ll 1$, which results in a curve with (one of more) distict peak(s) that is characteristic for the MUSIC pseudo spectrum.

An example of a MUSIC pseudo spectrum is plotted below:

![alt text](figures/MUSIC_pseudo_spectrum.png "MUSIC pseudo spectrum")

### MUSIC revisited: STFT processing for broadband signals
When MUSIC is applied to broadband signals via STFT-processing, a pseudo spectrum $\hat{P}_{MUSIC}(\theta, k)$ is calculated for each individual frequency bin $k$.

The individual MUSIC pseudo spectra $\hat{P}_{MUSIC}(\theta, k)$ are summed across all frequency bins $k \in \{1, 2, \ldots, K\}$.

In the simplest case, this is performed without normalization, i.e., 
$$\tilde{P}_{MUSIC}(\theta) = \sum_{k=1}^{K} \hat{P}_{MUSIC}(\theta, k)$$

While this is the commonly used implementation, it is far from being optimal, since
the maxima of the MUSIC pseudospectra $\hat{P}_{MUSIC}(\theta, k)$ may differ in orders of magnitude. By summing across frequencies without normalization, only the information of the few frequencies with the highest peaks in the MUSIC pseudo spectra are used. The information of frequencies with lower peaks in the MUSIC pseudo spectra  are practically not used to estimate the DOA.

To illustrate this, let's first plot the MUSIC pseudo spectra of 10 randomly selected frequency bins.


In [ ]:
fig = plt.figure(figsize=(14,10))
frequencies = sample(list(range(algorithms['MUSIC'].Pssl.shape[1])), k=10)
for i, k in enumerate(frequencies):
    plt.plot(algorithms["MUSIC"].Pssl[:,k])
plt.xlabel("angle [°]")
plt.title("Multiple narrowband MUSIC pseudo spectra in one plot", fontsize=15)


In the next cell, we calculate the maxima of the individual MUSIC pseudp-spectra per frequency bin $k$, i.e.,
$$ \hat{\theta}_k = max_\theta \;\hat{P}_{MUSIC}(\theta, k)$$
and visualize them in a swarm plot. 

From the plot, it should be clear that effectively only a few frequency bins contribute to the solution (if no normalization is applied).

In [ ]:
# calculation
maxima = sorted(np.max(algorithms["MUSIC"].Pssl, axis=0))

#plotting
fig, ax = plt.subplots(1, 1, figsize=(14,10))
sns.swarmplot(data=maxima, ax=ax, size=6)

ax.set_title("\nDistribution: Maxima of the MUSIC pseudo spectra of multiple frequency bins\n", fontsize=20)
ax.set_xticks([1])


To avoid the above mentioned problem, one could simply normalize the MUSIC pseudo spectra before summing across frequencies, i.e.,
$$\hat{P}_{NormMUSIC}(\theta, k) = \frac{\hat{P}_{MUSIC}(\theta, k)}{\hat{\theta}_k }$$
$$\tilde{P}_{NormMUSIC}(\theta) = \sum_{k=1}^{K} \hat{P}_{NormMUSIC}(\theta, k)$$



<a id='recommendation'></a>
## Recommendation

- As its performance is more robust, we recommend to use NormMUSIC over MUSIC
- When MUSIC is used as a baseline for publications, we recommend to include both MUSIC without frequency normalization and NormMUSIC because:
 - (i)   Using MUSIC without normalization ensures comparability with older papers.
 - (ii)  Using NormMUSIC ensures a fair comparison. Especially if you tweak the parameters of a new algorithm, you should not use a suboptimal implementation as a baseline.